In [ ]:
%load_ext autoreload
%autoreload 2

In [84]:
from imports import *
# from process_data import *
from train import initialize_model
from datasets import *
import matplotlib.pyplot as plt
import time
from torch_geometric.explain import Explainer
from torch_geometric.explain import ModelConfig
from torch_geometric.explain import ExplainerConfig
from torch_geometric.explain import DummyExplainer
from explain import *

import types
import networkx as nx

In [85]:
tiling_data = DNASeqGraphSeparateEdges("/Users/Brody1/Dropbox/Northwestern/DNA_Cyclizability/benchmarks/gnns", 
                                       raw_graph_name="cycle5", 
                                       processed_graph_name="cycle5_1_2_3_5_10_15_20_25_30_edge_separate_include_indices", 
                                       edge_distances=[1,2,3,5,10,15,20,25,30],
                                       include_indices=True)
# random_data = DNASeqGraphSeparateEdges("/Users/Brody1/Dropbox/Northwestern/DNA_Cyclizability/benchmarks/gnns", 
#                                        raw_graph_name="cycle3", 
#                                        processed_graph_name="cycle3_1_2_3_5_10_15_20_25_30_edge_separate_include_indices", 
#                                        edge_distances=[1,2,3,5,10,15,20,25,30],
#                                        include_indices=True)
# yeast_data = DNASeqGraphSeparateEdges("/Users/Brody1/Dropbox/Northwestern/DNA_Cyclizability/benchmarks/gnns", 
#                                       raw_graph_name="cycle1", 
#                                       processed_graph_name="cycle1_1_2_3_5_10_15_20_25_30_edge_separate_include_indices", 
#                                       edge_distances=[1,2,3,5,10,15,20,25,30],
#                                       include_indices=True)
# chrv_data = DNASeqGraphSeparateEdges("/Users/Brody1/Dropbox/Northwestern/DNA_Cyclizability/benchmarks/gnns", 
#                                      raw_graph_name="cycle6", 
#                                      processed_graph_name="cycle6_1_2_3_5_10_15_20_25_30_edge_separate_include_indices", 
#                                      edge_distances=[1,2,3,5,10,15,20,25,30],
#                                      include_indices=True)

In [86]:
cluster_list = [
    torch.Tensor(
        [0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,3,3,3,3,3,3,3,3,3,3,4,4,4,4,4,4,4,4,4,4])
]
hyperparameters = {"mp_hidden_channels":48, "mlp_hidden_channels":128, "num_layers":[1], "num_epochs":200, "learning_rate":0.0001, "batch_size":32,
                   "pool_ratios":[], "pool_types":[], "mlp_num_layers":2, "edge_lengths":[[1,2,3],[5,10,15,20,25,30]], 
                   "mp_layer":GCN, "jk":"lstm", "num_nodes":50, "max_time":8*60*60, "final_dropout": 0.2}

In [87]:
data_str = "tiling"

if data_str == "tiling":
    data = tiling_data
elif data_str == "random":
    data = random_data
elif data_str == "yeast":
    data = yeast_data
elif data_str == "chrv":
    data = chrv_data


num_obs=len(data)

In [88]:
avg_list = []
std_list = []
for i in range(4):
    avg_list.append(torch.mean(data.x[:,i]).item())
    std_list.append(torch.std(data.x[:,i]).item())

In [89]:
logging_info = {"print_every":1, "folder_path":"/Users/Brody1/Dropbox/Northwestern/DNA_Cyclizability/benchmarks/gnns/",
                "file_name": "edge_separate_1_2_3__5_10_15_20_25_30_with_mlps_2layer_GCN_1layer_48mphc_128mlphc_include_indices_no_final_pooling_02finaldropout_8hr"}

In [ ]:
model = initialize_model(model_name="GraphSAGESeparateEdgeLengthsWithMLPsNoFinalPooling", num_node_features=data.x.shape[1],
                         output_features=data.y.shape[1], hyperparameters=hyperparameters)
model.load_state_dict(torch.load(logging_info["folder_path"] + "models/" + logging_info["file_name"]))

In [91]:
# which_graph = np.random.random_integers(0, len(data), num_obs)
_, which_graph = torch.topk(data.y.flatten(), k=num_obs)
which_graph = np.array(which_graph)
# explanation_data = DataLoader(data[which_graph], batch_size=num_obs//10)
explanation_data = DataLoader(data[which_graph], batch_size=1)

In [ ]:
model.eval()
# predicted_values = []
# true_values = []
# for explanation_batch in explanation_data:
#     predicted_values.append(model(explanation_batch))
#     true_values.append(explanation_batch.y)
# # np.corrcoef((predicted_values[0].reshape(-1).detach().numpy(), true_values[0].reshape(-1).numpy()))[0,1]

# for i in range(len(predicted_values)):
#     for j in range(len(predicted_values[i])):
#         # print(f"i: {i}, j: {j}, i+j: {i+j}")
#         print(f"Index {i*len(predicted_values[i]) + j}:")
#         print(f"\t Predicted: {predicted_values[i][j].item()}, Actual: {true_values[i][j].item()}")

In [93]:
# # GraphSAGESeparateEdgeLengths

# def forward_until_mlps(model, data):
#     x, batch = data.x, data.batch
#     network_outputs = []

#     for network_idx, edge_length in enumerate(model.edge_lengths):

#         # edge_length_attr_name = f"edge_index_{edge_length}_locations"
#         edge_index_locations = abs(data.edge_index[0,:] - data.edge_index[1,:])==edge_length
#         edge_index_locations += (data.edge_index[0,:] - data.edge_index[1,:]) == 0
#         # edge_index_locations = getattr(data, edge_length_attr_name, None)
#         edge_index = data.edge_index[:,edge_index_locations]
#         if edge_index is None:
#             raise ValueError(f"Edge length of {edge_length} not found in data")
        
#         current_network = model.edge_length_networks[network_idx]

#         x_cur = current_network.convs[0](x=x, edge_index=edge_index, edge_weight=None)
#         batch_cur = batch

#         x_pooling = []
#         x_pooling.append(x_cur)

#         pooling_indices = []
#         pooling_edge_indices = []

#         for i, pooler in enumerate(current_network.poolers):
#             if current_network.pool_types[i] == "ASA":
#                 x_cur, edge_index, edge_weight, batch_cur, index = pooler(
#                     x=x_cur, edge_index=edge_index, batch=batch_cur)
#             else:
#                 x_cur, edge_index, edge_weight, batch_cur, index, score = pooler(
#                     x=x_cur, edge_index=edge_index, batch=batch_cur)
            
#             pooling_indices.append(index)
#             pooling_edge_indices.append(edge_index)
            
#             x_cur = current_network.convs[i+1](x=x_cur, edge_index=edge_index)
#             if i < len(current_network.poolers) - 1:
#                 x_pooling.append(x_cur)
        
#         for i, unpooler in enumerate(current_network.unpoolers):
#             unpooling_indices = pooling_indices.pop()
#             unpooling_edge_index = pooling_edge_indices.pop()
#             x_unpooling = torch.zeros(x_pooling[-1].shape)
#             x_unpooling[unpooling_indices] = x_cur
#             x_unpooling = unpooler(x=x_unpooling, edge_index=unpooling_edge_index)

#             x_unpooling = torch.cat((x_unpooling, x_pooling.pop()), dim=1)
            
#             x_cur = current_network.unpooling_mlps[i](x_unpooling)

#         network_outputs.append(current_network.final_mlp(x_cur))

#     return torch.cat(network_outputs, dim=1)

In [94]:
# # GraphSAGESeparateEdgeLengthsWithMLPs or GraphSAGESeparateEdgeLengthsWithMLPsNoFinalPooling

# def forward_until_mlps(model, data):
#     x, batch = data.x, data.batch
#     network_outputs = []

#     network_idx = 0

#     for outer_idx, edge_length_list in enumerate(model.edge_lengths):
#         inner_loop_network_outputs = []
#         for edge_length in edge_length_list:

#             # edge_length_attr_name = f"edge_index_{edge_length}_locations"
#             edge_index_locations = abs(data.edge_index[0,:] - data.edge_index[1,:])==edge_length
#             edge_index_locations += (data.edge_index[0,:] - data.edge_index[1,:]) == 0
#             # edge_index_locations = getattr(data, edge_length_attr_name, None)
#             edge_index = data.edge_index[:,edge_index_locations]
#             if edge_index is None:
#                 raise ValueError(f"Edge length of {edge_length} not found in data")
            
#             current_network = model.edge_length_networks[network_idx]

#             x_cur = current_network.convs[0](x=x, edge_index=edge_index, edge_weight=None)
#             batch_cur = batch

#             x_pooling = []
#             x_pooling.append(x_cur)

#             pooling_indices = []
#             pooling_edge_indices = []

#             for i, pooler in enumerate(current_network.poolers):
#                 if current_network.pool_types[i] == "ASA":
#                     x_cur, edge_index, edge_weight, batch_cur, index = pooler(
#                         x=x_cur, edge_index=edge_index, batch=batch_cur)
#                 else:
#                     x_cur, edge_index, edge_weight, batch_cur, index, score = pooler(
#                         x=x_cur, edge_index=edge_index, batch=batch_cur)
                
#                 pooling_indices.append(index)
#                 pooling_edge_indices.append(edge_index)
                
#                 x_cur = current_network.convs[i+1](x=x_cur, edge_index=edge_index)
#                 if i < len(current_network.poolers) - 1:
#                     x_pooling.append(x_cur)
            
#             for i, unpooler in enumerate(current_network.unpoolers):
#                 unpooling_indices = pooling_indices.pop()
#                 unpooling_edge_index = pooling_edge_indices.pop()
#                 x_unpooling = torch.zeros(x_pooling[-1].shape)
#                 x_unpooling[unpooling_indices] = x_cur
#                 x_unpooling = unpooler(x=x_unpooling, edge_index=unpooling_edge_index)

#                 x_unpooling = torch.cat((x_unpooling, x_pooling.pop()), dim=1)
                
#                 x_cur = current_network.unpooling_mlps[i](x_unpooling)

#             inner_loop_network_outputs.append(current_network.final_mlp(x_cur))

#             network_idx += 1

#         network_outputs.append(torch.cat(inner_loop_network_outputs, dim=1))

#     return torch.cat(network_outputs, dim=1)

In [95]:
# GraphSAGESeparateEdgeLengthsWithMLPs or GraphSAGESeparateEdgeLengthsWithMLPsNoFinalPooling

def forward_until_mlps(model, data):
    x, batch = data.x, data.batch
    network_outputs = []

    network_idx = 0

    for outer_idx, edge_length_list in enumerate(model.edge_lengths):
        inner_loop_network_outputs = []
        for edge_length in edge_length_list:

            # edge_length_attr_name = f"edge_index_{edge_length}_locations"
            edge_index_locations = abs(data.edge_index[0,:] - data.edge_index[1,:])==edge_length
            edge_index_locations += (data.edge_index[0,:] - data.edge_index[1,:]) == 0
            # edge_index_locations = getattr(data, edge_length_attr_name, None)
            edge_index = data.edge_index[:,edge_index_locations]
            if edge_index is None:
                raise ValueError(f"Edge length of {edge_length} not found in data")
            
            current_network = model.edge_length_networks[network_idx]

            x_cur = current_network.convs[0](x=x, edge_index=edge_index, edge_weight=None)
            batch_cur = batch

            x_pooling = []
            x_pooling.append(x_cur)

            pooling_indices = []
            pooling_edge_indices = []

            for i, pooler in enumerate(current_network.poolers):
                if current_network.pool_types[i] == "ASA":
                    x_cur, edge_index, edge_weight, batch_cur, index = pooler(
                        x=x_cur, edge_index=edge_index, batch=batch_cur)
                else:
                    x_cur, edge_index, edge_weight, batch_cur, index, score = pooler(
                        x=x_cur, edge_index=edge_index, batch=batch_cur)
                
                pooling_indices.append(index)
                pooling_edge_indices.append(edge_index)
                
                x_cur = current_network.convs[i+1](x=x_cur, edge_index=edge_index)
                if i < len(current_network.poolers) - 1:
                    x_pooling.append(x_cur)
            
            for i, unpooler in enumerate(current_network.unpoolers):
                unpooling_indices = pooling_indices.pop()
                unpooling_edge_index = pooling_edge_indices.pop()
                x_unpooling = torch.zeros(x_pooling[-1].shape)
                x_unpooling[unpooling_indices] = x_cur
                x_unpooling = unpooler(x=x_unpooling, edge_index=unpooling_edge_index)

                x_unpooling = torch.cat((x_unpooling, x_pooling.pop()), dim=1)
                
                x_cur = current_network.unpooling_mlps[i](x_unpooling)

            inner_loop_network_outputs.append(current_network.final_mlp(x_cur))

            network_idx += 1
        
        network_outputs.append(torch.cat(inner_loop_network_outputs, dim=1))

    return torch.cat(network_outputs, dim=1)

In [96]:
def max_pooling_x_by_cluster(x, batch, cluster_list):
    for cluster in cluster_list:
        cur_cluster = cluster.repeat(len(batch)//len(cluster)) + (torch.arange(len(batch))//len(cluster))*(torch.max(cluster)+1)
        x, batch = max_pool_x(cur_cluster, x, batch)
    return global_mean_pool(x, batch)

In [97]:
def edge_length_final_mlps(x, model, hidden_channels):
    outer_loop_network_outputs = []
    prev_idx = 0
    for outer_idx, edge_length_list in enumerate(model.edge_lengths):
        x_cur = x[:,prev_idx:(prev_idx+hidden_channels*len(edge_length_list))]
        outer_loop_network_outputs.append(model.edge_lengths_final_mlps[outer_idx](x_cur))
        prev_idx += hidden_channels*len(edge_length_list)
    return torch.cat(outer_loop_network_outputs, dim=1)

In [98]:
def final_mlps_no_pooling(x, model, batch):
    x = model.final_mlp_1(x)
    x = x.reshape(torch.max(batch)+1, -1)
    x = model.final_mlp_2(x)
    return x

In [99]:
# model_2 = tg_nn.Sequential('x, edge_index, batch, cluster_list', [(model.final_mlp_1, 'x -> x'),
#                                                     (max_pooling_x_by_cluster, 'x, batch, cluster_list -> x'),
#                                                     (model.final_mlp_2, 'x -> x')])

In [100]:
# model_2 = tg_nn.Sequential('x, edge_index, batch, cluster_list, orig_model, hidden_channels', [
#     (edge_length_final_mlps, 'x, orig_model, hidden_channels -> x'), 
#     (model.final_mlp_1, 'x -> x'),
#     (max_pooling_x_by_cluster, 'x, batch, cluster_list -> x'),
#     (model.final_mlp_2, 'x -> x')])

In [101]:
model_2 = tg_nn.Sequential('x, batch, orig_model, hidden_channels', [
    (edge_length_final_mlps, 'x, orig_model, hidden_channels -> x'), 
    (final_mlps_no_pooling, 'x, orig_model, batch -> x')])

In [102]:
model_config = ModelConfig(mode="regression", task_level="graph", return_type="raw")
explainer_config = ExplainerConfig(explanation_type="phenomenon", node_mask_type="attributes")
separate_network_explainer = CustomExplainer2(lr = .1)
separate_network_explainer.connect(explainer_config=explainer_config, model_config=model_config)

In [103]:
separate_network_explainer.epochs = 100

In [104]:
# 1, 5, 10, 15

# def explain_message(self, inputs: torch.Tensor, size_i: int) -> torch.Tensor:
#         # NOTE Replace this method in custom explainers per message-passing
#         # layer to customize how messages shall be explained, e.g., via:
#         # conv.explain_message = explain_message.__get__(conv, MessagePassing)
#         # see stackoverflow.com: 394770/override-a-method-at-instance-level

#      edge_mask = self._edge_mask

#      if edge_mask is None:
#           raise ValueError(f"Could not find a pre-defined 'edge_mask' as "
#                          f"part of {self.__class__.__name__}.")

#      if self._apply_sigmoid:
#           edge_mask = edge_mask.sigmoid()

#      # batch_size = num_obs
#      batch_size = 1
#      if inputs.size(self.node_dim) == 148*batch_size:
#           # Edge length 1
#           edge_mask = edge_mask[0:98*batch_size]
#      elif inputs.size(self.node_dim) == 140*batch_size:
#           # Edge length 5
#           edge_mask = edge_mask[98*batch_size:188*batch_size]
#      elif inputs.size(self.node_dim) == 130*batch_size:
#           # Edge length 10
#           edge_mask = edge_mask[188*batch_size:268*batch_size]
#      elif inputs.size(self.node_dim) == 120*batch_size:
#           # Edge length 15
#           edge_mask = edge_mask[268*batch_size:338*batch_size]

#      # Some ops add self-loops to `edge_index`. We need to do the same for
#      # `edge_mask` (but do not train these entries).
#      if inputs.size(self.node_dim) != edge_mask.size(0):
#           # edge_mask = edge_mask[self._loop_mask]
#           loop = edge_mask.new_ones(size_i)
#           edge_mask = torch.cat([edge_mask, loop], dim=0)
#      if inputs.size(self.node_dim) != edge_mask.size(0):
#           print(f"\t Inputs: {inputs.shape} \n \t Edge mask: {edge_mask.shape}")
#           assert inputs.size(self.node_dim) == edge_mask.size(0)

#      size = [1] * inputs.dim()
#      size[self.node_dim] = -1
#      return inputs * edge_mask.view(size)

In [105]:
# 1,2,3,5,10,15,20,25,30

def explain_message(self, inputs: torch.Tensor, size_i: int) -> torch.Tensor:
        # NOTE Replace this method in custom explainers per message-passing
        # layer to customize how messages shall be explained, e.g., via:
        # conv.explain_message = explain_message.__get__(conv, MessagePassing)
        # see stackoverflow.com: 394770/override-a-method-at-instance-level

    edge_mask = self._edge_mask

    if edge_mask is None:
        raise ValueError(f"Could not find a pre-defined 'edge_mask' as "
                         f"part of {self.__class__.__name__}.")

    if self._apply_sigmoid:
        edge_mask = edge_mask.sigmoid()

    # batch_size = num_obs//10
    batch_size = 1
    if inputs.size(self.node_dim) == 148*batch_size:
        # Edge length 1 (98)
        edge_mask = edge_mask[0:98*batch_size]
    if inputs.size(self.node_dim) == 146*batch_size:
        # Edge length 2 (96)
        edge_mask = edge_mask[98*batch_size:194*batch_size]
    if inputs.size(self.node_dim) == 144*batch_size:
        # Edge length 3 (94)
        edge_mask = edge_mask[194*batch_size:288*batch_size]
    elif inputs.size(self.node_dim) == 140*batch_size:
        # Edge length 5 (90)
        edge_mask = edge_mask[288*batch_size:378*batch_size]
    elif inputs.size(self.node_dim) == 130*batch_size:
        # Edge length 10 (80)
        edge_mask = edge_mask[378*batch_size:458*batch_size]
    elif inputs.size(self.node_dim) == 120*batch_size:
        # Edge length 15 (70)
        edge_mask = edge_mask[458*batch_size:528*batch_size]
    if inputs.size(self.node_dim) == 110*batch_size:
        # Edge length 20 (60)
        edge_mask = edge_mask[528*batch_size:588*batch_size]
    if inputs.size(self.node_dim) == 100*batch_size:
        # Edge length 25 (50)
        edge_mask = edge_mask[588*batch_size:638*batch_size]
    if inputs.size(self.node_dim) == 90*batch_size:
        # Edge length 30 (40)
        edge_mask = edge_mask[638*batch_size:678*batch_size]

     # Some ops add self-loops to `edge_index`. We need to do the same for
     # `edge_mask` (but do not train these entries).
    if inputs.size(self.node_dim) != edge_mask.size(0):
        # edge_mask = edge_mask[self._loop_mask]
        loop = edge_mask.new_ones(size_i)
        edge_mask = torch.cat([edge_mask, loop], dim=0)
    if inputs.size(self.node_dim) != edge_mask.size(0):
        print(f"\t Inputs: {inputs.shape} \n \t Edge mask: {edge_mask.shape}")
        assert inputs.size(self.node_dim) == edge_mask.size(0)

    size = [1] * inputs.dim()
    size[self.node_dim] = -1
    return inputs * edge_mask.view(size)

In [106]:
for i, module in enumerate(model.modules()):
    if hasattr(module, "explain_message"):
        # funcType = type(module.explain_message)
        # module.explain_message = funcType(explain_message2, module)
        module.explain_message = types.MethodType(explain_message, module)

In [107]:
# separate_network_explanations=[]
# for i, explanation_batch in enumerate(explanation_data):
#     until_mlps_out = forward_until_mlps(model, explanation_batch)

#     explanation_batch.x = until_mlps_out
#     model_2.eval()
#     separate_network_explanations.append(separate_network_explainer(model=model_2, data=explanation_batch, 
#                                                                     edge_index=torch.Tensor().reshape(2,-1), 
#                                                                     target=explanation_batch.y,
#                                                                     batch=explanation_batch.batch, 
#                                                                     cluster_list=model.cluster_list,
#                                                                     orig_model=model,
#                                                                     hidden_channels=hyperparameters["mlp_hidden_channels"]))
#     print(f"Completed batch {i}")

In [108]:
# separate_network_explanations=[]
# for i, explanation_batch in enumerate(explanation_data):
#     until_mlps_out = forward_until_mlps(model, explanation_batch)

#     explanation_batch.x = until_mlps_out
#     model_2.eval()
#     separate_network_explanations.append(separate_network_explainer(model=model_2, data=explanation_batch, 
#                                                                     target=explanation_batch.y,
#                                                                     batch=explanation_batch.batch, 
#                                                                     orig_model=model,
#                                                                     hidden_channels=hyperparameters["mlp_hidden_channels"]))
#     print(f"Completed batch {i}")

In [109]:
# hidden_channels = hyperparameters["mlp_hidden_channels"]
# all_network_contributions = []
# for network_explanations in separate_network_explanations:
#     # temp_explain = separate_network_explanations[0]
#     total_contribution = torch.sum(network_explanations.node_mask)
#     cur_network_contributions = []
#     for i in range(network_explanations.node_mask.shape[1]//hyperparameters["mlp_hidden_channels"]):
#         cur_network_contribution = torch.sum(network_explanations.node_mask[:,i*hidden_channels:(i+1)*hidden_channels])
#         cur_network_contributions.append(cur_network_contribution)
#         # print(f"Contribution from network corresponding to edge length {hyperparameters['edge_lengths'][i]}: {cur_network_contribution/total_contribution}")
#     all_network_contributions.append(cur_network_contributions)
# avg_network_contributions = []
# # for i in range(len(all_network_contributions[0])):
# #     avg_network_contributions.append(np.mean([all_network_contributions[j][i]/sum(all_network_contributions[j]) for j in range(len(all_network_contributions))]))
# #     print(f"Average contribution from network corresponding to edge length {hyperparameters['edge_lengths'][i]}: {avg_network_contributions[i]}")
# inner_idx_addition = 0
# for outer_idx, edge_length_list in enumerate(hyperparameters["edge_lengths"]):
#     for inner_idx, edge_length in enumerate(edge_length_list):
#         avg_network_contributions.append(np.mean([all_network_contributions[j][inner_idx_addition+inner_idx]/sum(all_network_contributions[j]) for j in range(len(all_network_contributions))]))
#         print(f"Average contribution from network corresponding to edge length {edge_length}: {avg_network_contributions[-1]}")
#     inner_idx_addition += len(edge_length_list)

In [110]:
model_config = ModelConfig(mode="regression", task_level="graph", return_type="raw")
explainer_config = ExplainerConfig(explanation_type="model", edge_mask_type="object")
# explainer_config = ExplainerConfig(explanation_type="phenomenon", node_mask_type="attributes")

In [111]:
test_explainer = CustomExplainer(lr = 0.1)
test_explainer.connect(explainer_config=explainer_config, model_config=model_config)
test_explainer.epochs=100

In [112]:
# test_explanations=[]
# for i, explanation_batch in enumerate(explanation_data):
#     test_explanations.append(test_explainer(model=model, data=explanation_batch, target=explanation_batch.y))
#     if (i+1) % 100 == 0:
#         torch.save(test_explanations, logging_info["folder_path"] + "explanations/" + logging_info["file_name"])
#         print(f"Completed batch {i}")
# torch.save(test_explanations, logging_info["folder_path"] + "explanations/" + logging_info["file_name"])

In [113]:
test_explanations = torch.load(logging_info["folder_path"] + "explanations/" + logging_info["file_name"] + "_" + data_str + "_full")

In [114]:
test_explanations2 = torch.load(logging_info["folder_path"] + "explanations/" + logging_info["file_name"] + "_" + data_str + "_TEMPORARY")

In [115]:
test_explanations = test_explanations + test_explanations2

In [116]:
if len(test_explanations) != num_obs:
    print(f"Resetting num obs: {len(test_explanations)}")
    num_obs = len(test_explanations)

In [117]:
# First find the average contribution for each batch:
batch_wise_processed_edge_weight_directed = []
for test_explanation in test_explanations:
    # Average of all weights in test_explanation
    # batch_wise_processed_edge_weight_directed.append(torch.mean(
    #     torch.stack([test_explanation.edge_mask[i*338:(i+1)*338] for i in range(test_explanation.edge_mask.shape[0]//338)]),
    #     dim=0))
    batch_wise_processed_edge_weight_directed.append(torch.mean(
        torch.stack([test_explanation.edge_mask[i*678:(i+1)*678] for i in range(test_explanation.edge_mask.shape[0]//678)]),
        dim=0))

batch_wise_processed_edge_weight_directed = torch.cat(batch_wise_processed_edge_weight_directed)

# processed_edge_weight_directed = torch.mean(
#     torch.stack([batch_wise_processed_edge_weight_directed[i*338:(i+1)*338] for i in range(batch_wise_processed_edge_weight_directed.shape[0]//338)]),
#     dim=0)
processed_edge_weight_directed = torch.mean(
    torch.stack([batch_wise_processed_edge_weight_directed[i*678:(i+1)*678] for i in range(batch_wise_processed_edge_weight_directed.shape[0]//678)]),
    dim=0)

processed_weighted_edges_directed = [(test_explanations[0].edge_index[0, i].item(), 
                                      test_explanations[0].edge_index[1, i].item(), 
                                      processed_edge_weight_directed[i].item()
                                      ) for i in range(len(processed_edge_weight_directed))]

In [118]:
processed_edge_weight = []
undirected_edges = []
prev_start = 0
# for i, edge_length in enumerate(hyperparameters["edge_lengths"]):
#     processed_edge_weight.append(torch.mean(torch.stack((
#         processed_edge_weight_directed[prev_start:prev_start+(50-edge_length)],
#         processed_edge_weight_directed[prev_start+(50-edge_length):prev_start+(50-edge_length)*2])),
#         dim=0))
#     undirected_edges.append(test_explanations[0].edge_index[:,prev_start:prev_start+(50-edge_length)])
#     prev_start += (50-edge_length)*2
i = 0
for edge_length_list in hyperparameters["edge_lengths"]:
    for edge_length in edge_length_list:
        processed_edge_weight.append(torch.mean(torch.stack((
            processed_edge_weight_directed[prev_start:prev_start+(50-edge_length)],
            processed_edge_weight_directed[prev_start+(50-edge_length):prev_start+(50-edge_length)*2])),
            dim=0))
        undirected_edges.append(test_explanations[0].edge_index[:,prev_start:prev_start+(50-edge_length)])
        prev_start += (50-edge_length)*2
        i+=1
processed_edge_weight = torch.cat(processed_edge_weight)
undirected_edges = torch.cat(undirected_edges, dim=1)

processed_weighted_edges = [(undirected_edges[0, i].item(), 
                             undirected_edges[1, i].item(), 
                             processed_edge_weight[i].item()
                             ) for i in range(len(processed_edge_weight))]

In [119]:
# processed_weighted_edges = []

# prev_start = 0
# for i, edge_length in enumerate(hyperparameters["edge_lengths"]):
#     print("This:")
#     print(test_explanations[0].edge_index[:,prev_start:prev_start+(50-edge_length)])
#     print("Should be the opposite of this:")
#     print(test_explanations[0].edge_index[:,prev_start+(50-edge_length):prev_start+(50-edge_length)*2])
#     prev_start += (50-edge_length)*2

In [ ]:
plt.figure(figsize=(14,12))

g = nx.Graph()

g.add_weighted_edges_from(processed_weighted_edges)
plotting_pos = nx.circular_layout(g)

edges,weights = zip(*nx.get_edge_attributes(g,'weight').items())
vmin = min(weights)
vmax = max(weights)
cmap=plt.cm.RdYlBu_r
plotting_labels = {node: str(node) for node in g.nodes()}

widths = [weight*20 for weight in weights]
alphas = [weight*3 for weight in weights]

nx.draw_networkx_labels(g, plotting_pos, labels=plotting_labels)
nx.draw_networkx_nodes(g,plotting_pos,label=plotting_labels)
edges = nx.draw_networkx_edges(g,plotting_pos,edge_color=weights,width=widths,
                               edge_cmap=cmap, alpha=alphas)
cbar = plt.colorbar(edges)
cbar.set_label("edge importance")
plt.axis('off')
plt.show()

In [ ]:
print(f"Average score for edge length 1: {torch.mean(processed_edge_weight[0:49]).item()}")
print(f"Average score for edge length 2: {torch.mean(processed_edge_weight[49:97]).item()}")
print(f"Average score for edge length 3: {torch.mean(processed_edge_weight[97:144]).item()}")
print(f"Average score for edge length 5: {torch.mean(processed_edge_weight[144:189]).item()}")
print(f"Average score for edge length 10: {torch.mean(processed_edge_weight[189:229]).item()}")
print(f"Average score for edge length 15: {torch.mean(processed_edge_weight[229:264]).item()}")
print(f"Average score for edge length 20: {torch.mean(processed_edge_weight[264:294]).item()}")
print(f"Average score for edge length 25: {torch.mean(processed_edge_weight[294:319]).item()}")
print(f"Average score for edge length 30: {torch.mean(processed_edge_weight[319:339]).item()}")

In [122]:
j=3

# If all in one batch:
# processed_edge_weight_directed_j = test_explanations[0].edge_mask[j*338:(j+1)*338]
# processed_edge_weight_directed_j = test_explanations[0].edge_mask[j*678:(j+1)*678]

# If all in separate batches:
processed_edge_weight_directed_j = test_explanations[j].edge_mask

# If 10 batches:
# processed_edge_weight_directed_j = test_explanations[j//10].edge_mask[(j//10)*338:((j//10)+1)*338]
# processed_edge_weight_directed_j = test_explanations[j//10].edge_mask[(j//10)*678:((j//10)+1)*678]

processed_weighted_edges_directed_j = [(test_explanations[0].edge_index[0, i].item(), 
                                        test_explanations[0].edge_index[1, i].item(), 
                                        processed_edge_weight_directed_j[i].item()
                                        ) for i in range(len(processed_edge_weight_directed_j))]

In [123]:
processed_edge_weight_j = []
undirected_edges_j = []
prev_start = 0
# for i, edge_length in enumerate(hyperparameters["edge_lengths"]):
#     processed_edge_weight_j.append(torch.mean(torch.stack((
#         processed_edge_weight_directed_j[prev_start:prev_start+(50-edge_length)],
#         processed_edge_weight_directed_j[prev_start+(50-edge_length):prev_start+(50-edge_length)*2])),
#         dim=0))
#     undirected_edges_j.append(test_explanations[0].edge_index[:,prev_start:prev_start+(50-edge_length)])
#     prev_start += (50-edge_length)*2
i=0
for edge_length_list in hyperparameters["edge_lengths"]:
    for edge_length in edge_length_list:
        processed_edge_weight_j.append(torch.mean(torch.stack((
            processed_edge_weight_directed_j[prev_start:prev_start+(50-edge_length)],
            processed_edge_weight_directed_j[prev_start+(50-edge_length):prev_start+(50-edge_length)*2])),
            dim=0))
        undirected_edges_j.append(test_explanations[0].edge_index[:,prev_start:prev_start+(50-edge_length)])
        prev_start += (50-edge_length)*2
        i+=1
processed_edge_weight_j = torch.cat(processed_edge_weight_j)
undirected_edges_j = torch.cat(undirected_edges_j, dim=1)

processed_weighted_edges_j = [(undirected_edges_j[0, i].item(), 
                               undirected_edges_j[1, i].item(), 
                               processed_edge_weight_j[i].item()
                               ) for i in range(len(processed_edge_weight_j))]

In [124]:
# processed_weighted_edges_j = [edge if edge[2] > 0.5 else (edge[0], edge[1], 0) for edge in processed_weighted_edges_j]

In [ ]:
plt.figure(figsize=(14,12))

g = nx.Graph()

g.add_weighted_edges_from(processed_weighted_edges_j)
plotting_pos = nx.circular_layout(g)

edges,weights = zip(*nx.get_edge_attributes(g,'weight').items())
vmin = min(weights)
vmax = max(weights)
cmap=plt.cm.RdYlBu_r

widths = [weight*5 for weight in weights]
alphas = [weight for weight in weights]
code = {0:"A", 1:"C", 2:"G", 3:"T"}
plotting_label_values = []
for i, intval in enumerate((data[which_graph][j].x[:,:4] == 1).nonzero(as_tuple=True)[1]):
    plotting_label_values.append(code[intval.item()])
plotting_labels = {node: plotting_label_values[i] for i,node in enumerate(g.nodes())}

node_color_dict = {"A": "red", "C": "green", "G": "blue", "T": "orange"}
node_colors = []
for label in plotting_label_values:
    node_colors.append(node_color_dict[label])

nx.draw_networkx_labels(g, plotting_pos, labels=plotting_labels)
nx.draw_networkx_nodes(g,plotting_pos,label=plotting_labels, node_color=node_colors)
edges = nx.draw_networkx_edges(g,plotting_pos,edge_color=weights,width=widths,
                               edge_cmap=cmap, alpha=alphas)
cbar = plt.colorbar(edges)
cbar.set_label("edge importance")
plt.axis('off')
plt.title(f"Sequence number {j}")
plt.show()

In [ ]:
processed_weighted_edges_j

In [127]:
# feature_labels = ["A", "C", "G", "T"]
# for position in range(50):
#     feature_labels.append(f"Position {position} index")
# test_explanations[0].visualize_feature_importance(feat_labels=feature_labels)

## K-means

In [128]:
from kmeans_pytorch import kmeans

In [129]:
kmeans_input = batch_wise_processed_edge_weight_directed.reshape(len(test_explanations), -1)

In [ ]:
num_clusters = 25
cluster_ids_x, cluster_centers = kmeans(X=kmeans_input, num_clusters=num_clusters, distance='euclidean', tol=0.000001)

In [131]:
kmeans_processed_edge_weight_directed_list = []
kmeans_processed_weighted_edges_directed_list = []
for j in range(num_clusters):
    cur_cluster_processed_edge_weight_directed = kmeans_input[cluster_ids_x == j]
    kmeans_processed_edge_weight_directed_list.append(
        torch.mean(cur_cluster_processed_edge_weight_directed, dim=0)
    )

    kmeans_processed_weighted_edges_directed_list.append(
        [(test_explanations[0].edge_index[0, i].item(), 
          test_explanations[0].edge_index[1, i].item(), 
          kmeans_processed_edge_weight_directed_list[-1][i].item()
          ) for i in range(len(kmeans_processed_edge_weight_directed_list[-1]))]
    )

In [132]:
kmeans_processed_edge_weight_list = []
kmeans_processed_weighted_edges_list = []

# for i, edge_length in enumerate(hyperparameters["edge_lengths"]):
#     processed_edge_weight.append(torch.mean(torch.stack((
#         processed_edge_weight_directed[prev_start:prev_start+(50-edge_length)],
#         processed_edge_weight_directed[prev_start+(50-edge_length):prev_start+(50-edge_length)*2])),
#         dim=0))
#     undirected_edges.append(test_explanations[0].edge_index[:,prev_start:prev_start+(50-edge_length)])
#     prev_start += (50-edge_length)*2
for cluster_id in range(num_clusters):
    i = 0
    prev_start = 0
    kmeans_processed_edge_weight_list.append([])
    undirected_edges = []
    for edge_length_list in hyperparameters["edge_lengths"]:
        for edge_length in edge_length_list:
            kmeans_processed_edge_weight_list[-1].append(torch.mean(torch.stack((
                kmeans_processed_edge_weight_directed_list[cluster_id][prev_start:prev_start+(50-edge_length)],
                kmeans_processed_edge_weight_directed_list[cluster_id][prev_start+(50-edge_length):prev_start+(50-edge_length)*2])),
                dim=0))
            undirected_edges.append(test_explanations[0].edge_index[:,prev_start:prev_start+(50-edge_length)])
            prev_start += (50-edge_length)*2
            i+=1
    kmeans_processed_edge_weight_list[-1] = torch.cat(kmeans_processed_edge_weight_list[-1])
    undirected_edges = torch.cat(undirected_edges, dim=1)

    kmeans_processed_weighted_edges_list.append(
        [(undirected_edges[0, i].item(), 
          undirected_edges[1, i].item(), 
          kmeans_processed_edge_weight_list[-1][i].item()
          ) for i in range(len(kmeans_processed_edge_weight_list[-1]))]
    )
    

In [ ]:
model = initialize_model(model_name="GraphSAGESeparateEdgeLengthsWithMLPsNoFinalPooling", num_node_features=data.x.shape[1],
                         output_features=data.y.shape[1], hyperparameters=hyperparameters)
model.load_state_dict(torch.load(logging_info["folder_path"] + "models/" + logging_info["file_name"]))
model.eval()

In [ ]:
x = [-3.5, 3.5]
y = [-3.5, 3.5]
code = {0:"A", 1:"C", 2:"G", 3:"T"}

true_y_by_cluster = []
pred_y_by_cluster = []

colors = ["b","g","r","c","m","y","tab:blue","tab:orange","tab:green","tab:red","tab:purple","tab:brown","tab:brown",
          "tab:pink","tab:gray","lightcoral","peru","gold","greenyellow","aquamarine","darkslategray","deepskyblue",
          "navy","deeppink","lavender"]

for cluster_id in range(num_clusters):

    # Network plot with weighted edges:
    g = nx.Graph()

    g.add_weighted_edges_from(kmeans_processed_weighted_edges_list[cluster_id])
    plotting_pos = nx.circular_layout(g)

    edges,weights = zip(*nx.get_edge_attributes(g,'weight').items())
    vmin = min(weights)
    vmax = max(weights)
    cmap=plt.cm.RdYlBu_r
    plotting_labels = {node: str(node) for node in g.nodes()}

    widths = [weight*20 for weight in weights]
    alphas = [weight for weight in weights]

    plt.figure(figsize=(12,10))
    nx.draw_networkx_labels(g, plotting_pos, labels=plotting_labels)
    nx.draw_networkx_nodes(g,plotting_pos,label=plotting_labels)
    
    edges = nx.draw_networkx_edges(g,plotting_pos,edge_color=weights,width=widths,
                                   edge_cmap=cmap, alpha=alphas)
    cbar = plt.colorbar(edges)
    cbar.set_label("edge importance")
    plt.axis('off')
    plt.title(f"Cluster number {cluster_id} ({kmeans_input[cluster_ids_x == cluster_id].shape[0]} sequences)")
    plt.show()

    # Network plot with most common nucleotide by position:
    cur_cluster_data = data[which_graph[0:len(test_explanations)][cluster_ids_x == cluster_id]]

    plotting_label_array = np.zeros((len(cur_cluster_data), 50))
    for i, cur_graph in enumerate(cur_cluster_data):
        for j, intval in enumerate((cur_graph.x[:,:4] == 1).nonzero(as_tuple=True)[1]):
            plotting_label_array[i, j] = intval.item()
    
    plotting_label_distr = np.zeros((50, 4))
    for i in range(4):
        plotting_label_distr[:,i] = np.sum(plotting_label_array==i, axis=0)/len(cur_cluster_data)

    plotting_label_values = [code[np.argmax(plotting_label_point)] for plotting_label_point in plotting_label_distr]
    plotting_label_pcts = np.round(np.max(plotting_label_distr, axis=1), 2)

    plotting_labels = {node: f"{plotting_label_values[i]}\n{plotting_label_pcts[i]}" for i,node in enumerate(g.nodes())}
    node_color_dict = {"A": "red", "C": "green", "G": "dodgerblue", "T": "orange"}
    node_colors = []
    for label in plotting_label_values:
        node_colors.append(node_color_dict[label])

    plt.figure(figsize=(12,10))
    nx.draw_networkx_labels(g, plotting_pos, labels=plotting_labels, font_size=8)
    nx.draw_networkx_nodes(g,plotting_pos, node_color=node_colors)
    
    edges = nx.draw_networkx_edges(g,plotting_pos,edge_color=weights,width=widths,
                                   edge_cmap=cmap, alpha=alphas)
    cbar = plt.colorbar(edges)
    cbar.set_label("edge importance")
    plt.axis('off')
    plt.title(f"Cluster number {cluster_id}, most common nucleotide ({kmeans_input[cluster_ids_x == cluster_id].shape[0]} sequences)")
    plt.show()

    # Network plot with 2 most common nucleotides by position:
    plotting_label_ids = [[arg_index for arg_index in np.argsort(-plotting_label_point)[:2]] for plotting_label_point in plotting_label_distr]
    plotting_label_values = [[code[plotting_label_id] for plotting_label_id in plotting_label_id_list] for plotting_label_id_list in plotting_label_ids]
    plotting_label_pcts = np.round([sum([plotting_label_point[plotting_label_id] for plotting_label_id in plotting_label_ids[i]]) for i, plotting_label_point in enumerate(plotting_label_distr)], 2)

    plotting_labels = {node: f"{plotting_label_values[i][0]}, {plotting_label_values[i][1]}\n{plotting_label_pcts[i]}" for i,node in enumerate(g.nodes())}

    plt.figure(figsize=(12,10))
    nx.draw_networkx_labels(g, plotting_pos, labels=plotting_labels, font_size=8)
    nx.draw_networkx_nodes(g,plotting_pos, node_color=node_colors)
    
    edges = nx.draw_networkx_edges(g,plotting_pos,edge_color=weights,width=widths,
                                   edge_cmap=cmap, alpha=alphas)
    cbar = plt.colorbar(edges)
    cbar.set_label("edge importance")
    plt.axis('off')
    plt.title(f"Cluster number {cluster_id}, 2 most common nucleotides ({kmeans_input[cluster_ids_x == cluster_id].shape[0]} sequences)")
    plt.show()


    # Network plot with highest deviation from mean nucleotide by position:
    plotting_label_distr2 = np.zeros((50, 4))
    for i in range(4):
        plotting_label_distr2[:,i] = (np.sum(plotting_label_array==i, axis=0)/len(cur_cluster_data) - avg_list[i])/std_list[i]

    plotting_label_values = [code[np.argmax(plotting_label_point)] for plotting_label_point in plotting_label_distr2]
    plotting_label_pcts = np.round(np.max(plotting_label_distr2, axis=1), 2)

    plotting_labels = {node: f"{plotting_label_values[i]}\n{plotting_label_pcts[i]}" for i,node in enumerate(g.nodes())}
    node_color_dict = {"A": "red", "C": "green", "G": "dodgerblue", "T": "orange"}
    node_colors = []
    for label in plotting_label_values:
        node_colors.append(node_color_dict[label])

    plt.figure(figsize=(12,10))
    nx.draw_networkx_labels(g, plotting_pos, labels=plotting_labels, font_size=8)
    nx.draw_networkx_nodes(g,plotting_pos, node_color=node_colors)
    
    edges = nx.draw_networkx_edges(g,plotting_pos,edge_color=weights,width=widths,
                                   edge_cmap=cmap, alpha=alphas)
    cbar = plt.colorbar(edges)
    cbar.set_label("edge importance")
    plt.axis('off')
    plt.title(f"Cluster number {cluster_id}, highest deviation from mean nucleotide ({kmeans_input[cluster_ids_x == cluster_id].shape[0]} sequences)")
    plt.show()



    print(f"Mean True C0 for cluster number {cluster_id}: {torch.mean(cur_cluster_data.y)}")

    cur_cluster_loader = DataLoader(cur_cluster_data, batch_size = 1)
    cur_cluster_pred = []
    for cur_cluster_batch in cur_cluster_loader:
        cur_cluster_pred.append(model(cur_cluster_batch).item())

    print(f"Mean Predicted C0 for cluster number {cluster_id}: {np.mean(cur_cluster_pred)}")

    # plt.hist(cur_cluster_data.y.flatten())
    # plt.title(f"Cluster number {cluster_id} True C0")
    # plt.xlim(x)
    # plt.show()

    # plt.hist(cur_cluster_pred)
    # plt.title(f"Cluster number {cluster_id} Predicted C0")
    # plt.xlim(x)
    # plt.show()

    plt.xlim(x)
    plt.ylim(y)
    plt.xlabel("Predicted")
    plt.ylabel("Observed")
    plt.plot(x, y, color="black")
    plt.scatter(cur_cluster_pred, cur_cluster_data.y.flatten(), marker="o", alpha=0.1)
    plt.title(f"Cluster number {cluster_id} Observed vs. Predicted C0")
    plt.show()

    true_y_by_cluster.append(cur_cluster_data.y.flatten())
    pred_y_by_cluster.append(cur_cluster_pred)

        # node_color_dict = {"A": "red", "C": "green", "G": "blue", "T": "orange"}

    plt.plot(plotting_label_distr[:,0], "red", label="A")
    plt.axhline(avg_list[0], color="red", alpha=0.5)
    plt.plot(plotting_label_distr[:,1], "green", label="C")
    plt.axhline(avg_list[1], color="green", alpha=0.5)
    plt.plot(plotting_label_distr[:,2], "dodgerblue", label="G")
    plt.axhline(avg_list[2], color="dodgerblue", alpha=0.5)
    plt.plot(plotting_label_distr[:,3], "orange", label="T")
    plt.axhline(avg_list[3], color="orange", alpha=0.5)
    plt.legend()
    plt.title(f"Cluster number {cluster_id} Nucleotide Content")
    plt.show()

    plt.plot(plotting_label_distr[:,0] + plotting_label_distr[:,3], "coral", label="A or T")
    plt.axhline(avg_list[0]+avg_list[3], color="coral", alpha=0.5)
    plt.plot(plotting_label_distr[:,1] + plotting_label_distr[:,2], "mediumaquamarine", label="C or G")
    plt.axhline(avg_list[1]+avg_list[2], color="mediumaquamarine", alpha=0.5)
    plt.legend()
    plt.title(f"Cluster number {cluster_id} Nucleotide Content, Grouped")
    plt.show()

plt.figure(figsize=(12,10))
plt.xlim(x)
plt.ylim(y)
plt.xlabel("Predicted")
plt.ylabel("Observed")
plt.plot(x, y, color="black")
for i in range(len(true_y_by_cluster)):
    plt.scatter(pred_y_by_cluster[i], true_y_by_cluster[i], marker="o", alpha=0.1, color=colors[i], label=f"Cluster {i}")
plt.legend()
plt.title(f"Observed vs. Predicted C0 By Cluster Assignment")
plt.show()

# Analysis of Individual Clusters

In [ ]:
x = [-3.5, 3.5]
y = [-3.5, 3.5]
code = {0:"A", 1:"C", 2:"G", 3:"T"}

cluster_id = 2
bases_of_interest = [1]
position_of_interest = 22

if len(bases_of_interest) == 1:
    bases_of_interest_str = code[bases_of_interest[0]]
else:
    bases_of_interest_str = ""
    for i, base_of_interest in enumerate(bases_of_interest):
        if i==0:
            bases_of_interest_str = code[base_of_interest]
        else: 
            bases_of_interest_str = bases_of_interest_str + ", " + code[base_of_interest]

g = nx.Graph()

g.add_weighted_edges_from(kmeans_processed_weighted_edges_list[cluster_id])
plotting_pos = nx.circular_layout(g)

edges,weights = zip(*nx.get_edge_attributes(g,'weight').items())
# weights = np.log(weights/(1-weights))
vmin = min(weights)
vmax = max(weights)
cmap=plt.cm.RdYlBu_r
plotting_labels = {node: str(node) for node in g.nodes()}

widths = [weight*20 for weight in weights]
alphas = [weight for weight in weights]

plt.figure(figsize=(12,10))
nx.draw_networkx_labels(g, plotting_pos, labels=plotting_labels)
nx.draw_networkx_nodes(g,plotting_pos,label=plotting_labels)

edges = nx.draw_networkx_edges(g,plotting_pos,edge_color=weights,width=widths,
                                edge_cmap=cmap, alpha=alphas)
cbar = plt.colorbar(edges)
cbar.set_label("edge importance")
plt.axis('off')
plt.title(f"Cluster number {cluster_id} ({kmeans_input[cluster_ids_x == cluster_id].shape[0]} sequences)")
plt.show()

cur_cluster_data = tiling_data[which_graph[0:len(test_explanations)][cluster_ids_x == cluster_id]]

plotting_label_array = np.zeros((len(cur_cluster_data), 50))
for i, cur_graph in enumerate(cur_cluster_data):
    for j, intval in enumerate((cur_graph.x[:,:4] == 1).nonzero(as_tuple=True)[1]):
        plotting_label_array[i, j] = intval.item()

plotting_label_distr = np.zeros((50, 4))
for i in range(4):
    plotting_label_distr[:,i] = np.sum(plotting_label_array==i, axis=0)/len(cur_cluster_data)

plotting_label_values = [code[np.argmax(plotting_label_point)] for plotting_label_point in plotting_label_distr]
plotting_label_pcts = np.round(np.max(plotting_label_distr, axis=1), 2)

plotting_labels = {node: f"{plotting_label_values[i]}\n{plotting_label_pcts[i]}" for i,node in enumerate(g.nodes())}
node_color_dict = {"A": "red", "C": "green", "G": "blue", "T": "orange"}
node_colors = []
for label in plotting_label_values:
    node_colors.append(node_color_dict[label])

plt.figure(figsize=(12,10))
nx.draw_networkx_labels(g, plotting_pos, labels=plotting_labels, font_size=8)
nx.draw_networkx_nodes(g,plotting_pos, node_color=node_colors)

edges = nx.draw_networkx_edges(g,plotting_pos,edge_color=weights,width=widths,
                                edge_cmap=cmap, alpha=alphas)
cbar = plt.colorbar(edges)
cbar.set_label("edge importance")
plt.axis('off')
plt.title(f"Cluster number {cluster_id}, most common nucleotide ({kmeans_input[cluster_ids_x == cluster_id].shape[0]} sequences)")
plt.show()

plotting_label_ids = [[arg_index for arg_index in np.argsort(-plotting_label_point)[:2]] for plotting_label_point in plotting_label_distr]
plotting_label_values = [[code[plotting_label_id] for plotting_label_id in plotting_label_id_list] for plotting_label_id_list in plotting_label_ids]
plotting_label_pcts = np.round([sum([plotting_label_point[plotting_label_id] for plotting_label_id in plotting_label_ids[i]]) for i, plotting_label_point in enumerate(plotting_label_distr)], 2)

plotting_labels = {node: f"{plotting_label_values[i][0]}, {plotting_label_values[i][1]}\n{plotting_label_pcts[i]}" for i,node in enumerate(g.nodes())}

plt.figure(figsize=(12,10))
nx.draw_networkx_labels(g, plotting_pos, labels=plotting_labels, font_size=8)
nx.draw_networkx_nodes(g,plotting_pos, node_color=node_colors)

edges = nx.draw_networkx_edges(g,plotting_pos,edge_color=weights,width=widths,
                                edge_cmap=cmap, alpha=alphas)
cbar = plt.colorbar(edges)
cbar.set_label("edge importance")
plt.axis('off')
plt.title(f"Cluster number {cluster_id}, 2 most common nucleotides ({kmeans_input[cluster_ids_x == cluster_id].shape[0]} sequences)")
plt.show()

print(f"Mean True C0 for cluster number {cluster_id}: {torch.mean(cur_cluster_data.y)}")

plt.plot(plotting_label_distr[:,0], "red", label="A")
plt.plot(plotting_label_distr[:,1], "green", label="C")
plt.plot(plotting_label_distr[:,2], "blue", label="G")
plt.plot(plotting_label_distr[:,3], "orange", label="T")
plt.legend()
plt.title(f"Cluster number {cluster_id} Nucleotide Content")
plt.show()


of_interest_0_graphs = []
of_interest_0_true_y = []

of_interest_1_graphs = []
of_interest_1_true_y = []
for graph in cur_cluster_data:
    of_interest = False
    for base_of_interest in bases_of_interest:
        if graph.x[position_of_interest, base_of_interest] == 1:
            of_interest = True
    if of_interest:
        of_interest_1_graphs.append(graph)
        of_interest_1_true_y.append(graph.y.item())
    else:
        of_interest_0_graphs.append(graph)
        of_interest_0_true_y.append(graph.y.item())

print(f"Mean True C0 for cluster number {cluster_id} when position {position_of_interest} is {bases_of_interest_str}: {np.mean(of_interest_1_true_y)}")

cur_cluster_loader = DataLoader(of_interest_1_graphs, batch_size = 1)
cur_cluster_pred = []
for cur_cluster_batch in cur_cluster_loader:
    cur_cluster_pred.append(model(cur_cluster_batch).item())

print(f"Mean Predicted C0 for cluster number {cluster_id} when position {position_of_interest} is {bases_of_interest_str}: {np.mean(cur_cluster_pred)}")

plt.xlim(x)
plt.ylim(y)
plt.xlabel("Predicted")
plt.ylabel("Observed")
plt.plot(x, y, color="black")
plt.scatter(cur_cluster_pred, of_interest_1_true_y, marker="o", alpha=0.1)
plt.title(f"Cluster number {cluster_id} Observed vs. Predicted C0 when position {position_of_interest} is {bases_of_interest_str}")
plt.show()

of_interest_1_plotting_label_array = np.zeros((len(of_interest_1_graphs), 50))
for i, cur_graph in enumerate(of_interest_1_graphs):
    for j, intval in enumerate((cur_graph.x[:,:4] == 1).nonzero(as_tuple=True)[1]):
        of_interest_1_plotting_label_array[i, j] = intval.item()

of_interest_1_plotting_label_distr = np.zeros((50, 4))
for i in range(4):
    of_interest_1_plotting_label_distr[:,i] = np.sum(of_interest_1_plotting_label_array==i, axis=0)/len(of_interest_1_graphs)

plt.plot(of_interest_1_plotting_label_distr[:,0], "red", label="A")
plt.plot(of_interest_1_plotting_label_distr[:,1], "green", label="C")
plt.plot(of_interest_1_plotting_label_distr[:,2], "blue", label="G")
plt.plot(of_interest_1_plotting_label_distr[:,3], "orange", label="T")
plt.legend()
plt.title(f"Cluster number {cluster_id} Nucleotide Content when position {position_of_interest} is {bases_of_interest_str}")
plt.show()

print(f"Mean True C0 for cluster number {cluster_id} when position {position_of_interest} is not {bases_of_interest_str}: {np.mean(of_interest_0_true_y)}")

cur_cluster_loader = DataLoader(of_interest_0_graphs, batch_size = 1)
cur_cluster_pred = []
for cur_cluster_batch in cur_cluster_loader:
    cur_cluster_pred.append(model(cur_cluster_batch).item())

print(f"Mean Predicted C0 for cluster number {cluster_id} when position {position_of_interest} is not {bases_of_interest_str}: {np.mean(cur_cluster_pred)}")

plt.xlim(x)
plt.ylim(y)
plt.xlabel("Predicted")
plt.ylabel("Observed")
plt.plot(x, y, color="black")
plt.scatter(cur_cluster_pred, of_interest_0_true_y, marker="o", alpha=0.1)
plt.title(f"Cluster number {cluster_id} Observed vs. Predicted C0 when position {position_of_interest} is not {bases_of_interest_str}")
plt.show()

of_interest_0_plotting_label_array = np.zeros((len(of_interest_0_graphs), 50))
for i, cur_graph in enumerate(of_interest_0_graphs):
    for j, intval in enumerate((cur_graph.x[:,:4] == 1).nonzero(as_tuple=True)[1]):
        of_interest_0_plotting_label_array[i, j] = intval.item()

of_interest_0_plotting_label_distr = np.zeros((50, 4))
for i in range(4):
    of_interest_0_plotting_label_distr[:,i] = np.sum(of_interest_0_plotting_label_array==i, axis=0)/len(of_interest_0_graphs)

plt.plot(of_interest_0_plotting_label_distr[:,0], "red", label="A")
plt.plot(of_interest_0_plotting_label_distr[:,1], "green", label="C")
plt.plot(of_interest_0_plotting_label_distr[:,2], "blue", label="G")
plt.plot(of_interest_0_plotting_label_distr[:,3], "orange", label="T")
plt.legend()
plt.title(f"Cluster number {cluster_id} Nucleotide Content when position {position_of_interest} is not {bases_of_interest_str}")
plt.show()


In [157]:
# of_interest_0_true_y = []
# of_interest_1_true_y = []
# for graph in cur_cluster_data:
#     of_interest = False
#     for base_of_interest in bases_of_interest:
#         if graph.x[position_of_interest, base_of_interest] == 1:
#             of_interest = True
#     if of_interest:
#         of_interest_0_true_y.append(graph.y.item())
#     else:
#         of_interest_1_true_y.append(graph.y.item())

In [ ]:
len(of_interest_0_true_y + of_interest_1_true_y)

In [212]:
# outlier_cluster_id = 16 # Actual Outlier Cluster
outlier_cluster_id = 3 # Highest Average C0 Cluster
outlier_cluster_data = data[which_graph[0:len(test_explanations)][cluster_ids_x == outlier_cluster_id]]

In [ ]:
outlier_cluster_processed_edge_weight_directed = kmeans_input[cluster_ids_x == outlier_cluster_id]

for j in range(len(outlier_cluster_data)):

    processed_edge_weight_directed_j = outlier_cluster_processed_edge_weight_directed[j]


    processed_weighted_edges_directed_j = [(test_explanations[0].edge_index[0, i].item(), 
                                            test_explanations[0].edge_index[1, i].item(), 
                                            processed_edge_weight_directed_j[i].item()
                                            ) for i in range(len(processed_edge_weight_directed_j))]
    
    processed_edge_weight_j = []
    undirected_edges_j = []
    prev_start = 0

    i=0
    for edge_length_list in hyperparameters["edge_lengths"]:
        for edge_length in edge_length_list:
            processed_edge_weight_j.append(torch.mean(torch.stack((
                processed_edge_weight_directed_j[prev_start:prev_start+(50-edge_length)],
                processed_edge_weight_directed_j[prev_start+(50-edge_length):prev_start+(50-edge_length)*2])),
                dim=0))
            undirected_edges_j.append(test_explanations[0].edge_index[:,prev_start:prev_start+(50-edge_length)])
            prev_start += (50-edge_length)*2
            i+=1
    processed_edge_weight_j = torch.cat(processed_edge_weight_j)
    undirected_edges_j = torch.cat(undirected_edges_j, dim=1)

    processed_weighted_edges_j = [(undirected_edges_j[0, i].item(), 
                                undirected_edges_j[1, i].item(), 
                                processed_edge_weight_j[i].item()
                                ) for i in range(len(processed_edge_weight_j))]
    
    # plt.figure(figsize=(12,10))
    plt.figure(figsize=(10,10))

    g = nx.Graph()

    g.add_weighted_edges_from(processed_weighted_edges_j)
    plotting_pos = nx.circular_layout(g)

    edges,weights = zip(*nx.get_edge_attributes(g,'weight').items())
    vmin = min(weights)
    vmax = max(weights)
    cmap=plt.cm.RdYlBu_r

    widths = [weight*5 for weight in weights]
    alphas = [weight for weight in weights]
    code = {0:"A", 1:"C", 2:"G", 3:"T"}
    plotting_label_values = []
    for i, intval in enumerate((outlier_cluster_data[j].x[:,:4] == 1).nonzero(as_tuple=True)[1]):
        plotting_label_values.append(code[intval.item()])
    plotting_labels = {node: plotting_label_values[i] for i,node in enumerate(g.nodes())}

    node_color_dict = {"A": "red", "C": "green", "G": "dodgerblue", "T": "orange"}
    node_colors = []
    for label in plotting_label_values:
        node_colors.append(node_color_dict[label])

    nx.draw_networkx_labels(g, plotting_pos, labels=plotting_labels)
    nx.draw_networkx_nodes(g,plotting_pos,label=plotting_labels, node_color=node_colors)
    # edges = nx.draw_networkx_edges(g,plotting_pos,edge_color=weights,width=widths,
    #                             edge_cmap=cmap, alpha=alphas)
    # cbar = plt.colorbar(edges)
    # cbar.set_label("edge importance")
    plt.axis('off')
    plt.title(f"Cluster {outlier_cluster_id}, Sequence number {j}")
    plt.show()

    print(f"Cluster {outlier_cluster_id}, Sequence number {j} true C0: {round(true_y_by_cluster[outlier_cluster_id][j].item(),2)}")
    print(f"Cluster {outlier_cluster_id}, Sequence number {j} predicted C0: {round(pred_y_by_cluster[outlier_cluster_id][j], 2)}")

In [169]:
from scipy.stats import f_oneway

In [170]:
cluster_data = pd.DataFrame({
    'cluster_assignment': cluster_ids_x,
    'C0': data.y[which_graph[range(len(test_explanations))]].flatten()
})

In [ ]:
plt.hist(cluster_data.C0, bins=20)

In [ ]:
# Perform ANOVA
anova_result = f_oneway(
    cluster_data[cluster_data['cluster_assignment'] == 0]['C0'],
    cluster_data[cluster_data['cluster_assignment'] == 1]['C0'],
    cluster_data[cluster_data['cluster_assignment'] == 2]['C0'],
    cluster_data[cluster_data['cluster_assignment'] == 3]['C0'],
    cluster_data[cluster_data['cluster_assignment'] == 4]['C0'],
    cluster_data[cluster_data['cluster_assignment'] == 5]['C0'],
    cluster_data[cluster_data['cluster_assignment'] == 6]['C0'],
    cluster_data[cluster_data['cluster_assignment'] == 7]['C0'],
    cluster_data[cluster_data['cluster_assignment'] == 8]['C0'],
    cluster_data[cluster_data['cluster_assignment'] == 9]['C0'],
    cluster_data[cluster_data['cluster_assignment'] == 10]['C0'],
    cluster_data[cluster_data['cluster_assignment'] == 11]['C0'],
    cluster_data[cluster_data['cluster_assignment'] == 12]['C0'],
    cluster_data[cluster_data['cluster_assignment'] == 13]['C0'],
    cluster_data[cluster_data['cluster_assignment'] == 14]['C0'],
    cluster_data[cluster_data['cluster_assignment'] == 15]['C0'],
    cluster_data[cluster_data['cluster_assignment'] == 16]['C0'],
    cluster_data[cluster_data['cluster_assignment'] == 17]['C0'],
    cluster_data[cluster_data['cluster_assignment'] == 18]['C0'],
    cluster_data[cluster_data['cluster_assignment'] == 19]['C0'],
    cluster_data[cluster_data['cluster_assignment'] == 20]['C0'],
    cluster_data[cluster_data['cluster_assignment'] == 21]['C0'],
    cluster_data[cluster_data['cluster_assignment'] == 22]['C0'],
    cluster_data[cluster_data['cluster_assignment'] == 23]['C0'],
    cluster_data[cluster_data['cluster_assignment'] == 24]['C0'],
)

# Display ANOVA results
print("ANOVA p-value:", anova_result.pvalue)

# Check the significance
alpha = 0.05
if anova_result.pvalue < alpha:
    print("Reject the null hypothesis. There is a significant difference between clusters.")
else:
    print("Fail to reject the null hypothesis. There is no significant difference between clusters.")

In [ ]:
df_between = len(cluster_data['cluster_assignment'].unique()) - 1
df_total = len(cluster_data) - 1
df_resid = df_total - df_between

eta_squared = anova_result.statistic / (anova_result.statistic + df_resid)
print("Eta-squared:", eta_squared)

In [209]:
import seaborn as sns

In [ ]:
mean_values = cluster_data.groupby('cluster_assignment')['C0'].mean().sort_values(ascending=False).index
cluster_data['cluster_assignment'] = cluster_data['cluster_assignment'].astype(pd.CategoricalDtype(categories=mean_values, ordered=True))

plt.figure(figsize=(10, 6))
sns.boxplot(x='cluster_assignment', y='C0', data=cluster_data)
plt.title('Side-by-Side Boxplots of C0 by Cluster')
plt.xlabel('Cluster')
plt.ylabel('Output Variable')
plt.show()

# Sequence-Space Sub-Clustering

In [ ]:
sub_cluster_ids_x_list = []
for cluster_id in range(num_clusters):
    cur_cluster_kmeans_input = data[which_graph[0:len(test_explanations)][cluster_ids_x == cluster_id]].x[:,:4].reshape(sum(cluster_ids_x == cluster_id), -1)
    # cur_cluster_ids_x, cur_cluster_centers = kmeans(X=cur_cluster_kmeans_input, num_clusters=2, distance='euclidean', tol=0.000000001)
    cur_cluster_ids_x, cur_cluster_centers = kmeans(X=cur_cluster_kmeans_input, num_clusters=2, distance='euclidean')
    sub_cluster_ids_x_list.append(cur_cluster_ids_x)

In [ ]:
for cluster_id in range(num_clusters):
    cur_cluster_data = data[which_graph[0:len(test_explanations)][cluster_ids_x == cluster_id]]
    cur_cluster_sub_cluster0_data = cur_cluster_data[sub_cluster_ids_x_list[cluster_id] == 0]
    cur_cluster_sub_cluster1_data = cur_cluster_data[sub_cluster_ids_x_list[cluster_id] == 1]

    print(f"Cluster number {cluster_id} overall mean C0: {round(torch.mean(cur_cluster_data.y).item(), 2)}")
    print(f"\t Sub-cluster 0 mean C0: {round(torch.mean(cur_cluster_sub_cluster0_data.y).item(), 2)}")
    print(f"\t Sub-cluster 1 mean C0: {round(torch.mean(cur_cluster_sub_cluster1_data.y).item(), 2)}")
    print(f"\t Absolute difference: {round(abs(torch.mean(cur_cluster_sub_cluster0_data.y) - torch.mean(cur_cluster_sub_cluster1_data.y)).item(), 2)}")

    plt.xlim(x)
    plt.ylim(y)
    plt.xlabel("Predicted")
    plt.ylabel("Observed")
    plt.plot(x, y, color="black")
    plt.scatter(torch.Tensor(pred_y_by_cluster[cluster_id])[sub_cluster_ids_x_list[cluster_id] == 0], 
                true_y_by_cluster[cluster_id][sub_cluster_ids_x_list[cluster_id] == 0], 
                marker="o", alpha=0.1, label="Subcluster 0", color="r")
    plt.scatter(torch.Tensor(pred_y_by_cluster[cluster_id])[sub_cluster_ids_x_list[cluster_id] == 1], 
                true_y_by_cluster[cluster_id][sub_cluster_ids_x_list[cluster_id] == 1], 
                marker="o", alpha=0.1, label="Subcluster 1", color="b")
    plt.title(f"Cluster number {cluster_id} Observed vs. Predicted C0 By Subcluster")
    plt.legend()
    plt.show()

    # plt.xlim(x)
    # plt.ylim(y)
    # plt.xlabel("Predicted")
    # plt.ylabel("Observed")
    # plt.plot(x, y, color="black")
    # plt.title(f"Cluster number {cluster_id} Subcluster 1 Observed vs. Predicted C0")
    # plt.show()

    

# Start at smallest:

In [ ]:
processed_edge_weight = []
undirected_edges = []
prev_start = 0
# for i, edge_length in enumerate(hyperparameters["edge_lengths"]):
#     processed_edge_weight.append(torch.mean(torch.stack((
#         processed_edge_weight_directed[prev_start:prev_start+(50-edge_length)],
#         processed_edge_weight_directed[prev_start+(50-edge_length):prev_start+(50-edge_length)*2])),
#         dim=0))
#     undirected_edges.append(test_explanations[0].edge_index[:,prev_start:prev_start+(50-edge_length)])
#     prev_start += (50-edge_length)*2
i = 0
for edge_length_list in hyperparameters["edge_lengths"]:
    for edge_length in edge_length_list:
        processed_edge_weight.append(torch.mean(torch.stack((
            processed_edge_weight_directed[prev_start:prev_start+(50-edge_length)],
            processed_edge_weight_directed[prev_start+(50-edge_length):prev_start+(50-edge_length)*2])),
            dim=0))
        undirected_edges.append(test_explanations[0].edge_index[:,prev_start:prev_start+(50-edge_length)])
        prev_start += (50-edge_length)*2
        i+=1
processed_edge_weight = torch.cat(processed_edge_weight)
undirected_edges = torch.cat(undirected_edges, dim=1)

processed_weighted_edges = [(undirected_edges[0, i].item(), 
                             undirected_edges[1, i].item(), 
                             processed_edge_weight[i].item()
                             ) for i in range(len(processed_edge_weight))]

In [ ]:
cluster_ids_x

In [567]:
# which_graph = np.random.random_integers(0, len(data), num_obs)
_, which_graph_least = torch.topk(data.y.flatten(), k=num_obs, largest=False)
which_graph_least = np.array(which_graph_least)
explanation_data_least = DataLoader(data[which_graph_least], batch_size=1)

In [ ]:
model.eval()
predicted_values_least = []
true_values_least = []
for explanation_batch in explanation_data_least:
    predicted_values_least.append(model(explanation_batch))
    true_values_least.append(explanation_batch.y)
# np.corrcoef((predicted_values[0].reshape(-1).detach().numpy(), true_values[0].reshape(-1).numpy()))[0,1]

for i in range(len(predicted_values_least)):
    for j in range(len(predicted_values_least[i])):
        # print(f"i: {i}, j: {j}, i+j: {i+j}")
        print(f"Index {i*len(predicted_values_least[i]) + j}:")
        print(f"\t Predicted: {predicted_values_least[i][j].item()}, Actual: {true_values_least[i][j].item()}")

In [569]:
model_config = ModelConfig(mode="regression", task_level="graph", return_type="raw")
explainer_config = ExplainerConfig(explanation_type="phenomenon", node_mask_type="attributes")
separate_network_explainer_least = CustomExplainer2(lr = .1)
separate_network_explainer_least.connect(explainer_config=explainer_config, model_config=model_config)

In [570]:
separate_network_explainer_least.epochs = 1000

In [571]:
separate_network_explanations_least=[]
for explanation_batch in explanation_data_least:
    until_mlps_out = forward_until_mlps(model, explanation_batch)

    explanation_batch.x = until_mlps_out
    model_2.eval()
    separate_network_explanations_least.append(separate_network_explainer_least(model=model_2, data=explanation_batch, 
                                                                    edge_index=torch.Tensor().reshape(2,-1), 
                                                                    target=explanation_batch.y,
                                                                    batch=explanation_batch.batch, 
                                                                    cluster_list=model.cluster_list))

In [ ]:
hidden_channels = hyperparameters["hidden_channels"]
all_network_contributions_least = []
for network_explanations in separate_network_explanations_least:
    # temp_explain = separate_network_explanations[0]
    total_contribution = torch.sum(network_explanations.node_mask)
    cur_network_contributions = []
    for i in range(network_explanations.node_mask.shape[1]//hyperparameters["hidden_channels"]):
        cur_network_contribution = torch.sum(network_explanations.node_mask[:,i*hidden_channels:(i+1)*hidden_channels])
        cur_network_contributions.append(cur_network_contribution)
        # print(f"Contribution from network corresponding to edge length {hyperparameters['edge_lengths'][i]}: {cur_network_contribution/total_contribution}")
    all_network_contributions_least.append(cur_network_contributions)
avg_network_contributions_least = []
for i in range(len(all_network_contributions_least[0])):
    avg_network_contributions_least.append(np.mean([all_network_contributions_least[j][i]/sum(all_network_contributions_least[j]) for j in range(len(all_network_contributions_least))]))
    print(f"Average contribution from network corresponding to edge length {hyperparameters['edge_lengths'][i]}: {avg_network_contributions_least[i]}")

In [578]:
model_config = ModelConfig(mode="regression", task_level="graph", return_type="raw")
explainer_config = ExplainerConfig(explanation_type="model", edge_mask_type="object")

In [579]:
test_explainer_least = CustomExplainer(lr = 0.01)
test_explainer_least.connect(explainer_config=explainer_config, model_config=model_config)
test_explainer_least.epochs=1000

In [581]:
test_explanations_least=[]
for explanation_batch in explanation_data_least:
    test_explanations_least.append(test_explainer_least(model=model, data=explanation_batch, target=explanation_batch.y))

In [582]:
# First find the average contribution for each batch:
batch_wise_processed_edge_weight_directed_least = []
for test_explanation in test_explanations_least:
    # Average of all weights in test_explanation
    batch_wise_processed_edge_weight_directed_least.append(torch.mean(
        torch.stack([test_explanation.edge_mask[i*338:(i+1)*338] for i in range(test_explanation.edge_mask.shape[0]//338)]),
        dim=0))

batch_wise_processed_edge_weight_directed_least = torch.cat(batch_wise_processed_edge_weight_directed_least)

processed_edge_weight_directed_least = torch.mean(
    torch.stack([batch_wise_processed_edge_weight_directed_least[i*338:(i+1)*338] for i in range(batch_wise_processed_edge_weight_directed_least.shape[0]//338)]),
    dim=0)

processed_weighted_edges_directed_least = [(test_explanations_least[0].edge_index[0, i].item(), 
                                            test_explanations_least[0].edge_index[1, i].item(), 
                                            processed_edge_weight_directed_least[i].item()
                                            ) for i in range(len(processed_edge_weight_directed))]

In [583]:
processed_edge_weight_least = []
undirected_edges_least = []
prev_start = 0
for i, edge_length in enumerate(hyperparameters["edge_lengths"]):
    processed_edge_weight_least.append(torch.mean(torch.stack((
        processed_edge_weight_directed_least[prev_start:prev_start+(50-edge_length)],
        processed_edge_weight_directed_least[prev_start+(50-edge_length):prev_start+(50-edge_length)*2])),
        dim=0))
    undirected_edges_least.append(test_explanations_least[0].edge_index[:,prev_start:prev_start+(50-edge_length)])
    prev_start += (50-edge_length)*2
processed_edge_weight_least = torch.cat(processed_edge_weight_least)
undirected_edges_least = torch.cat(undirected_edges_least, dim=1)

processed_weighted_edges_least = [(undirected_edges_least[0, i].item(), 
                                   undirected_edges_least[1, i].item(), 
                                   processed_edge_weight_least[i].item()
                                   ) for i in range(len(processed_edge_weight_least))]

In [ ]:
plt.figure(figsize=(14,12))

g = nx.Graph()

g.add_weighted_edges_from(processed_weighted_edges_least)
plotting_pos = nx.circular_layout(g)

edges,weights = zip(*nx.get_edge_attributes(g,'weight').items())
vmin = min(weights)
vmax = max(weights)
cmap=plt.cm.coolwarm
plotting_labels = {node: str(node) for node in g.nodes()}

widths = [weight*5 for weight in weights]

nx.draw_networkx_labels(g, plotting_pos, labels=plotting_labels)
nx.draw_networkx_nodes(g,plotting_pos,label=plotting_labels)
edges = nx.draw_networkx_edges(g,plotting_pos,edge_color=weights,width=widths,
                               edge_cmap=cmap)
cbar = plt.colorbar(edges)
cbar.set_label("edge importance")
plt.axis('off')
plt.show()

In [ ]:
processed_weighted_edges_least

In [611]:
j=57

# If all in one batch:
# processed_edge_weight_directed_j = test_explanations[0].edge_mask[j*338:(j+1)*338]

# If all in separate batches:
processed_edge_weight_directed_j_least = test_explanations_least[j].edge_mask

processed_weighted_edges_directed_j_least = [(test_explanations_least[0].edge_index[0, i].item(), 
                                              test_explanations_least[0].edge_index[1, i].item(), 
                                              processed_edge_weight_directed_j_least[i].item()
                                              ) for i in range(len(processed_edge_weight_directed_j_least))]

In [612]:
processed_edge_weight_j_least = []
undirected_edges_j_least = []
prev_start = 0
for i, edge_length in enumerate(hyperparameters["edge_lengths"]):
    processed_edge_weight_j_least.append(torch.mean(torch.stack((
        processed_edge_weight_directed_j_least[prev_start:prev_start+(50-edge_length)],
        processed_edge_weight_directed_j_least[prev_start+(50-edge_length):prev_start+(50-edge_length)*2])),
        dim=0))
    undirected_edges_j_least.append(test_explanations_least[0].edge_index[:,prev_start:prev_start+(50-edge_length)])
    prev_start += (50-edge_length)*2
processed_edge_weight_j_least = torch.cat(processed_edge_weight_j_least)
undirected_edges_j_least = torch.cat(undirected_edges_j_least, dim=1)

processed_weighted_edges_j_least = [(undirected_edges_j_least[0, i].item(), 
                                     undirected_edges_j_least[1, i].item(), 
                                     processed_edge_weight_j_least[i].item()
                                     ) for i in range(len(processed_edge_weight_j_least))]

In [617]:
processed_weighted_edges_j_least = [edge if edge[2] > 0.7 else (edge[0], edge[1], 0) for edge in processed_weighted_edges_j_least]

In [ ]:
plt.figure(figsize=(14,12))

g = nx.Graph()

g.add_weighted_edges_from(processed_weighted_edges_j_least)
plotting_pos = nx.circular_layout(g)

edges,weights = zip(*nx.get_edge_attributes(g,'weight').items())
vmin = min(weights)
vmax = max(weights)
cmap=plt.cm.coolwarm

widths = [weight*5 for weight in weights]
code = {0:"A", 1:"C", 2:"G", 3:"T"}
plotting_label_values = []
for i, intval in enumerate((data[which_graph_least][j].x[:,:4] == 1).nonzero(as_tuple=True)[1]):
    plotting_label_values.append(code[intval.item()])
plotting_labels = {node: plotting_label_values[i] for i,node in enumerate(g.nodes())}

node_color_dict = {"A": "red", "C": "green", "G": "blue", "T": "orange"}
node_colors = []
for label in plotting_label_values:
    node_colors.append(node_color_dict[label])

nx.draw_networkx_labels(g, plotting_pos, labels=plotting_labels)
nx.draw_networkx_nodes(g,plotting_pos,label=plotting_labels, node_color=node_colors)
edges = nx.draw_networkx_edges(g,plotting_pos,edge_color=weights,width=widths,
                               edge_cmap=cmap)
cbar = plt.colorbar(edges)
cbar.set_label("edge importance")
plt.axis('off')
plt.title(f"Sequence number {j} (Least)")
plt.show()